In [1]:
!pip uninstall -y pacmap && pip install git+https://github.com/YingfanWang/PaCMAP
!pip install git+https://github.com/erichson/optht/
!pip uninstall -y eonacs && pip install  git+https://github.com/njnmco/smc #--log /dev/stderr

  Cloning https://github.com/YingfanWang/PaCMAP to /tmp/pip-req-build-juer3j9w
  Running command git clone -q https://github.com/YingfanWang/PaCMAP /tmp/pip-req-build-juer3j9w
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 646 kB 8.0 MB/s 
  Created wheel for pacmap: filename=pacmap-0.6.2-py3-none-any.whl size=17848 sha256=6178e437aa3e5881be4b636592bbcfb67ede6b55a05cb0cb847ab79fa461f179
  Stored in directory: /tmp/pip-ephem-wheel-cache-67sh4bba/wheels/d3/ba/bb/572a4340fc753b41ae48d4ed40a283777147ecbf9dfe71ff96
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391678 sha256=691f6d2a9e7575588c65796a7c0b41fef6bfa9669382e8d4e084b749b1d65092
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built pacmap annoy
  Cloning https://github.com/erichson/optht/ to /tmp/pip-req-buil

In [2]:
import eonacs.common.util as util

util.colab_map_drive()


Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

In [ ]:
# tasks = pd.read_sql("select distinct substr(onetsoc_code, 1, 2) as soc,  task as text from task_statements",   con)


In [4]:
import eonacs.common.reduction as reduction


/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [ ]:
# import eonacs.common.dbert as dbert
# import torch
# from transformers import AutoModel
# tokenizer, model = dbert.dbert()
# fine  = AutoModel.from_pretrained("/content/drive/MyDrive/smc/models/dbert_tasks/")
# hyper = AutoModel.from_pretrained("/content/drive/MyDrive/smc/models/dbert_tasks_hyper/")

In [ ]:
# hyper_e = dbert.get_embeddings(tasks['text'], tokenizer, hyper)

In [ ]:
# pd.to_pickle((tasks,hyper_e), "/content/drive/MyDrive/smc/data/tasks_dbert_hyper_e.pkl.gz") 

In [5]:
tasks, hyper_e = pd.read_pickle("/content/drive/MyDrive/smc/data/tasks_dbert_hyper_e.pkl.gz")

In [ ]:
s = np.linalg.svd(hyper_e, compute_uv=False)

In [ ]:
import optht
optht.optht(hyper_e, s)

259

In [6]:
from sklearn.preprocessing import StandardScaler
features = StandardScaler().fit_transform(hyper_e)

In [ ]:
# %%time 
# m = reduction.PaCMAP2(120)
# m.fit(features)

In [ ]:
# dim 2 => 16s
# 5 => 23s
# 20 => 41
# 50 => 61
# 120 => 97 (cpu 163)
# 260 => 103

In [14]:
import pacmap

In [17]:
import multiprocessing

In [11]:
def cv(features, nfolds, loss):
  folds = np.random.randint(0, nfolds, features.shape[0])
  def score_fold(i):
    train, test = features[folds != i, :], features[folds == i, :]
    m = pacmap.PaCMAP(10)
    pred = m.fit(train).transform(test, basis=train)
    return loss(test, pred)
  return [score_fold(i) for i in range(nfolds)]

In [39]:
  def score_fold_par(ARGS):
    i, features, folds, loss = ARGS 
    train, test = features[folds != i, :], features[folds == i, :]
    m = pacmap.PaCMAP(10)
    pred = m.fit(train).transform(test, basis=train)
    return loss(test, pred)


In [66]:
def cv_par(features, nfolds, loss):
  folds = np.random.randint(0, nfolds, features.shape[0])
  ARGS = zip(range(10), (features,)*10, (folds,)*10, (loss,)*10)

  with multiprocessing.Pool(5) as pool:
    return pool.map(score_fold_par, ARGS)

In [24]:
import scipy

def loss(Y, Yhat):
  test_dist = scipy.spatial.distance.pdist(Y)
  pred_dist = scipy.spatial.distance.pdist(Yhat)
  return 1 - scipy.stats.spearmanr(test_dist, pred_dist).correlation

In [ ]:
%time losses = cv(features, 10, loss)
losses

In [ ]:
%time losses = cv_par(features, 10, loss)
losses

In [16]:
sorted(losses)[1] # d = 10

0.5368049716813366

In [48]:
import os
os.cpu_count()

2

In [ ]:
sorted(losses)[1] # d = 2

0.6131274957075751

In [19]:
    m = pacmap.PaCMAP(10)
    pred = m.fit(features).transform(features, features)


In [64]:
import time
from datetime import datetime

def f(i):
  start = datetime.now()
  time.sleep(i)
  end = datetime.now()
  return start, end


with multiprocessing.Pool(10) as p:
  %time print(p.map(f, range(20)))

with multiprocessing.Pool(10) as p:
  %time print(p.map(f, reversed(range(20))))

[(datetime.datetime(2022, 4, 14, 20, 26, 43, 688734), datetime.datetime(2022, 4, 14, 20, 26, 43, 688787)), (datetime.datetime(2022, 4, 14, 20, 26, 43, 693909), datetime.datetime(2022, 4, 14, 20, 26, 44, 695044)), (datetime.datetime(2022, 4, 14, 20, 26, 43, 689960), datetime.datetime(2022, 4, 14, 20, 26, 45, 692106)), (datetime.datetime(2022, 4, 14, 20, 26, 43, 693699), datetime.datetime(2022, 4, 14, 20, 26, 46, 696877)), (datetime.datetime(2022, 4, 14, 20, 26, 43, 694017), datetime.datetime(2022, 4, 14, 20, 26, 47, 698189)), (datetime.datetime(2022, 4, 14, 20, 26, 43, 694009), datetime.datetime(2022, 4, 14, 20, 26, 48, 697500)), (datetime.datetime(2022, 4, 14, 20, 26, 43, 693826), datetime.datetime(2022, 4, 14, 20, 26, 49, 699981)), (datetime.datetime(2022, 4, 14, 20, 26, 43, 693945), datetime.datetime(2022, 4, 14, 20, 26, 50, 701109)), (datetime.datetime(2022, 4, 14, 20, 26, 43, 693368), datetime.datetime(2022, 4, 14, 20, 26, 51, 697704)), (datetime.datetime(2022, 4, 14, 20, 26, 43, 6